https://radimrehurek.com/gensim/auto_examples/howtos/run_doc2vec_imdb.html

https://radimrehurek.com/gensim/models/doc2vec.html

## Configurations

In [ ]:
%matplotlib inline

In [ ]:
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

In [ ]:
pip install testfixtures

In [ ]:
import collections

SentimentDocument = collections.namedtuple('SentimentDocument', 'words tags split sentiment')

In [ ]:
inputFileName = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Processed-Merged-Dataset-99-08-28.csv'


outputLabelledTTDS= '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Balenced-Dataset-00-06-26.csv'
outputDocsFileName = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/DocsReadyForD2V-00-06-26.csv'

d2vModelFNmPos = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelPositive-00-06-26.model'
d2vModelFNtPos = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelPositive-00-06-26.txt'

d2vModelFNmNeg = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelNegative-00-06-26.model'
d2vModelFNtNeg = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelNegative-00-06-26.txt'

csvTopic8 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_8.csv'
csvTopic20 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_20.csv'
csvTopic40 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_40.csv'
csvTopic80 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_80.csv'
csvTopic50 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_50.csv'
csvTopic60 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_60.csv'
csvTopic65 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_65.csv'
csvTopic70 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_70.csv'


d2vModelFNm0 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel0-00-06-23.model'
d2vModelFNt0 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel0-00-06-23.txt'

d2vModelFNm1 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel1-00-06-23.model'
d2vModelFNt1 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel1-00-06-23.txt'

d2vModelFNm2 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel2-00-06-23.model'
d2vModelFNt2 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel2-00-06-23.txt'

d2vModelFNm3 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel3-00-06-23.model'
d2vModelFNt3 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel3-00-06-23.txt'

csvTopic8MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_8_MostSimilar_cs.csv'
csvTopic20MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_20_MostSimilar_cs.csv'
csvTopic40MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_40_MostSimilar_cs.csv'
csvTopic50MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_50_MostSimilar_cs.csv'
csvTopic60MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_60_MostSimilar_cs.csv'
csvTopic65MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_65_MostSimilar_cs.csv'
csvTopic70MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_70_MostSimilar_cs.csv'
csvTopic80MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByCosine/Topics_80_MostSimilar_cs.csv'



In [ ]:
def create_sentiment_document(index, tex, sent, sp):
    #_, split, sentiment_str, _ = name.split('/')
    #sentiment = {'pos': 1.0, 'neg': 0.0, 'unsup': None}[sentiment_str]

    #if sentiment is None:
        #split = 'extra'
    #print(tex)
    tokens = gensim.utils.to_unicode(tex).split()
    #tokens = gensim.utils.sp
    return SentimentDocument(tokens, [index], sp, sent)

# Colab

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Prepair Dataset (Dividing into Test and Train)

In [ ]:
dfReviews = pd.DataFrame()
dfReviews = pd.read_csv(inputFileName, encoding = 'utf-8', header = None )
dfReviews.head(3)

In [ ]:
inputFileName

In [ ]:
dfReviews.shape

In [ ]:
dfReviews = dfReviews.dropna()

In [ ]:
dfReviews.shape

In [ ]:
len (dfReviews.loc[dfReviews[3] == 0])

In [ ]:
len (dfReviews.loc[dfReviews[3] == 1])

In [ ]:
dfReviews[4] = 'train'

In [ ]:
dfReviews.columns

In [ ]:
dfReviews.shape

In [ ]:
testNumber=113320
trainNumber=453280

In [ ]:
#Making a copy of All REVIEWS
dfReviews2 = dfReviews

In [ ]:
#Selecting 500,000 records from alol reviews (250,000 positive and 250,000 negative) to make a balanced dataset
dfTemp1 = dfReviews[dfReviews[3] == 1].sample( n = 283300)
dfTemp2 = dfReviews[dfReviews[3] == 0].sample( n = 283300)
dfTemp = dfTemp1.append(dfTemp2)
print( "length of dfTemp " + str( len (dfTemp) ))
print(len(dfTemp[dfTemp[3]==0]))
print(len(dfTemp[dfTemp[3]==1]))
dfReviews = dfTemp
dfTest = dfTemp[dfTemp[3] == 0].sample( n = 56660)
dfTest = dfTest.append(dfTemp[dfTemp[3] == 1].sample( n = 56660))
print( "length of dfTest " + str( len (dfTest) ) )
print(len(dfTest[dfTest[3]==0]))
print(len(dfTest[dfTest[3]==1]))

print( "length of dfReviews " + str( len (dfReviews) ) )
print(len(dfReviews[dfReviews[3]==0]))
print(len(dfReviews[dfReviews[3]==1]))

In [ ]:
dfTest.head(3)

In [ ]:
dfReviews.shape

In [ ]:
len (dfReviews.loc[dfReviews[4] == 'train'])

In [ ]:
len (dfReviews.loc[dfReviews[4] == 'test'])

In [ ]:
dfReviews.head()

In [ ]:
len(dfTest)

In [ ]:
testRows = dfTest[1]
type(testRows)

In [ ]:
testRows

In [ ]:
len(testRows)

In [ ]:
type(dfReviews)

# Preparing test data

In [ ]:
dfReviews.loc[dfReviews[1].isin(testRows),4] = 'test'

In [ ]:
dfReviews.shape

In [ ]:
len (dfReviews.loc[dfReviews[4] == 'test'])

In [ ]:
len (dfReviews.loc[dfReviews[4] == 'train'])

In [ ]:
print( "length of dfReviews " + str( len (dfReviews) ) )
print(len(dfReviews[dfReviews[3]==0]))
print(len(dfReviews[dfReviews[3]==1]))

In [ ]:
dfReviews.iloc[238937,4]

In [ ]:
print( "length of dfReviews " + str( len (dfReviews) ) )
print(len(dfReviews[dfReviews[3]==0]))
print(len(dfReviews[dfReviews[3]==1]))

In [ ]:
print( "length of dfReviews " + str( len (dfReviews) ) )
print(len(dfReviews[dfReviews[4]=='test']))
print(len(dfReviews[dfReviews[4]=='train']))

In [ ]:
dfReviews.sample(n=5)

In [ ]:
dfReviews.reset_index(drop=True, inplace=True)

In [ ]:
dfReviews.shape

In [ ]:
dfReviews.head(3)

In [ ]:
dfReviews.drop(columns=[0,1], inplace= True)

In [ ]:
dfReviews.columns = ['Text', 'Sentiment', 'Type']
dfReviews.head(3)

In [ ]:
print( "length of dfReviews " + str( len (dfReviews) ) )
print(len(dfReviews[dfReviews['Sentiment']==0]))
print(len(dfReviews[dfReviews['Sentiment']==1]))

# Save the divided dataset into csv file





In [ ]:
dfReviews.to_csv( outputLabelledTTDS, header=True, index=True)

In [ ]:
outputLabelledTTDS

In [ ]:
dfReviews.sample(5)

## Load Saved LabelledTTDS

In [ ]:
#dfReviews = pd.read_csv(outputLabelledTTDS, header=None)

In [ ]:
len(dfReviews)

In [ ]:
dfReviews.columns

# Make the big list of sentiment and doc to be learned by Doc2Vec

In [ ]:
allDocs = list()

In [ ]:
dfReviews.sample(10)

In [ ]:
i=0
for index , row in dfReviews.iterrows():
  allDocs.append(  create_sentiment_document( (index) , (row["Text"]) , (row["Sentiment"]) , row["Type"] ) )
  #allDocs.append(  create_sentiment_document( (index) , (row[2]) , (row[3]) , row[4] ) )
  i = i + 1

In [ ]:
len(allDocs)

In [ ]:
allDocs[500]

In [ ]:
dfReviews.loc[500]

In [ ]:
len(allDocs)

In [ ]:
allDocs[400000]

## Writing all docs to file

In [ ]:
my_df = pd.DataFrame(allDocs)
my_df.to_csv(outputDocsFileName,  header=True, index = True)

In [ ]:
outputDocsFileName

# Make, Train, and Save Two doc2vec models

In [ ]:
trainPositiveDocs = [doc for doc in allDocs if doc.split == 'train' and doc.sentiment == 1 ]
testPositiveDocs = [doc for doc in allDocs if doc.split == 'test' and doc.sentiment == 1]

trainNegativeDocs = [doc for doc in allDocs if doc.split == 'train' and doc.sentiment == 0]
testNegativeDocs = [doc for doc in allDocs if doc.split == 'test' and doc.sentiment == 0]

In [ ]:
allPositiveDocs = [doc for doc in allDocs if  doc.sentiment == 1 ]
allNegativeDocs = [doc for doc in allDocs if  doc.sentiment == 0 ]

In [ ]:
type(allDocs)

In [ ]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

d2vPositiveModel = Doc2Vec(dm=0, dm_mean = 1,  vector_size=400, epochs=20, min_count=5,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0)

d2vNegativeModel = Doc2Vec(dm=0, dm_mean = 1,  vector_size=400, epochs=20, min_count=5,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0)

d2vPositiveModel.build_vocab(allPositiveDocs)
print("%s vocabulary scanned & state initialized" % d2vPositiveModel)

d2vNegativeModel.build_vocab(allNegativeDocs)
print("%s vocabulary scanned & state initialized" % d2vNegativeModel)

#models_by_name = OrderedDict((str(model), model) for model in simple_models)

In [ ]:
from random import shuffle
shuffled_positiveDocs = allPositiveDocs[:]
shuffled_negativeDocs = allNegativeDocs[:]

shuffle(shuffled_positiveDocs)
shuffle(shuffled_negativeDocs)

In [ ]:
print(len(shuffled_positiveDocs))
print(len(shuffled_negativeDocs))


In [ ]:
    print("Training %s" % d2vPositiveModel)
    d2vPositiveModel.train(shuffled_positiveDocs, total_examples=len(shuffled_positiveDocs), epochs=10)

    print("Training %s" % d2vNegativeModel)
    d2vNegativeModel.train(shuffled_negativeDocs, total_examples=len(shuffled_negativeDocs), epochs=10)

    d2vNegativeModel.save(d2vModelFNmNeg)
    d2vNegativeModel.save(d2vModelFNtNeg)

    d2vPositiveModel.save(d2vModelFNmPos)
    d2vPositiveModel.save(d2vModelFNtPos)




## Are inferred vectors close to the precalculated ones?

In [ ]:
doc_id = np.random.randint(d2vPositiveModel.docvecs.count)  # Pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)

In [ ]:
allDocs[4]

In [ ]:
doc_id = np.random.randint(d2vPositiveModel.docvecs.count)
print(allDocs[doc_id])
print(doc_id)

In [ ]:
inferred_docvec = d2vPositiveModel.infer_vector(allDocs[doc_id].words)
print('%s:\n %s' % (d2vPositiveModel, d2vPositiveModel.docvecs.most_similar([inferred_docvec], topn=3)))

In [ ]:
allDocs[429632]

# Find similarities

## Loading Saved d2v Model

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

In [ ]:
d2vPositiveModel = Doc2Vec.load(d2vModelFNtPos)
d2vNegativeModel = Doc2Vec.load(d2vModelFNtNeg)


# Find the most similar docs tn = 8




In [ ]:
dfTopic8 = pd.read_csv(csvTopic8)
print(dfTopic8.shape)
dfTopic8 = dfTopic8.dropna()
dfSimTopic8 = pd.DataFrame()
keywordsTopic8 = []

keywordsTopic8 = dfTopic8['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic8 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  #totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  #totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))

  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())


#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives
print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll8 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll8.to_csv(csvTopic8MostSimilar, header=True)



# Visualization

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)

In [ ]:
dfSimTopicPos[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
dfSimTopic1[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.4, stop=1, step=0.1)
dfSimTopic3[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
dfAll8 = dfSimTopicNeg.append(dfSimTopicPos)
print(len(dfAll8))

In [ ]:
csvTopic8MostSimilar

In [ ]:
dfAll8.to_csv(csvTopic8MostSimilar, header=True)




# Find the most similar docs tn = 20

In [ ]:


dfTopic20 = pd.read_csv(csvTopic20)
print(dfTopic20.shape)
dfTopic20 = dfTopic20.dropna()
dfSimTopic20 = pd.DataFrame()
keywordsTopic20= []

keywordsTopic20= dfTopic20['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic20:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))



  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll8 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll8.to_csv(csvTopic20MostSimilar, header=True)




# Visualizatiom

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)

In [ ]:
dfSimTopic0_20[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
dfSimTopic1_20[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_20[1].hist(bins=bin_values, figsize=[8,4])


In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic3_20[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 40

In [ ]:


dfTopic40 = pd.read_csv(csvTopic40)
print(dfTopic40.shape)
dfTopic40 = dfTopic40.dropna()
dfSimTopic40 = pd.DataFrame()
keywordsTopic40= []

keywordsTopic40= dfTopic40['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic40:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))
  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll8 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll8.to_csv(csvTopic40MostSimilar, header=True)




# Visuaization

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_40[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic1_40[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_40[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic3_40[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 50

In [ ]:
dfTopic50 = pd.read_csv(csvTopic50)
print(dfTopic50.shape)
dfTopic50 = dfTopic50.dropna()
dfSimTopic50 = pd.DataFrame()
keywordsTopic50= []

keywordsTopic50= dfTopic50['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic50:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))

  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll50 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll50.to_csv(csvTopic50MostSimilar, header=True)




# Visualization

In [ ]:

bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_50[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic1_50[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_50[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic3_50[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 60

In [ ]:

dfTopic60 = pd.read_csv(csvTopic60)
print(dfTopic60.shape)
dfTopic60 = dfTopic60.dropna()
dfSimTopic60 = pd.DataFrame()
keywordsTopic60= []

keywordsTopic60= dfTopic60['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic60:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))
  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll60 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll60.to_csv(csvTopic60MostSimilar, header=True)




# Visualization

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_60[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic1_60[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_60[1].hist(bins=bin_values, figsize=[8,4])

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic3_60[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 65

In [ ]:


dfTopic65 = pd.read_csv(csvTopic65)
print(dfTopic65.shape)
dfTopic65 = dfTopic65.dropna()
dfSimTopic65 = pd.DataFrame()
keywordsTopic65= []

keywordsTopic65= dfTopic65['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic65:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))

  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll65 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll65.to_csv(csvTopic65MostSimilar, header=True)




# Visualization

In [ ]:

bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_65[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
dfSimTopic1_65[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_65[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
dfSimTopic3_65[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 70

In [ ]:
csvTopic70MostSimilar

In [ ]:


dfTopic70 = pd.read_csv(csvTopic70)
print(dfTopic70.shape)
dfTopic70 = dfTopic70.dropna()
dfSimTopic70 = pd.DataFrame()
keywordsTopic70= []

keywordsTopic70= dfTopic70['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic70:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))

  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll70 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll70.to_csv(csvTopic70MostSimilar, header=True)




# Visualization

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_70[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
dfSimTopic1_70[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_70[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
dfSimTopic3_70[1].hist(bins=bin_values, figsize=[8,4])

# Find the most similar docs tn = 80

In [ ]:
csvTopic80MostSimilar

In [ ]:


dfTopic80 = pd.read_csv(csvTopic80)
print(dfTopic80.shape)
dfTopic80 = dfTopic80.dropna()
dfSimTopic80 = pd.DataFrame()
keywordsTopic80= []

keywordsTopic80= dfTopic80['Keywords']

dfSimTopicPos = pd.DataFrame()
totalVectorPos = np.zeros(400)

dfSimTopicNeg = pd.DataFrame()
totalVectorNeg = np.zeros(400)



for item in keywordsTopic80:
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  jtems = item.split(',')
  inferred_docvecPos = d2vPositiveModel.infer_vector(jtems)
  totalVectorPos = totalVectorPos + inferred_docvecPos
  inferred_docvecNeg = d2vNegativeModel.infer_vector(jtems)
  totalVectorNeg = totalVectorNeg + inferred_docvecNeg
  dfSimTopicPos = dfSimTopicPos.append( d2vPositiveModel.docvecs.most_similar([inferred_docvecPos], topn=50000))
  dfSimTopicNeg = dfSimTopicNeg.append( d2vNegativeModel.docvecs.most_similar([inferred_docvecNeg], topn=50000))

  #end of for


dfSimTopicPos.columns = ["docid","SimilarityScore"]
dfSimTopicNeg.columns = ["docid","SimilarityScore"]

print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives

print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos["SimilarityScore"]))
print("len Negatives:" , len(dfSimTopicNeg["SimilarityScore"]))

dfAll80 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll80.to_csv(csvTopic80MostSimilar, header=True)




# Visualization

In [ ]:
bin_values = np.arange(start=0.5, stop=1, step=0.1)
dfSimTopic0_80[1].hist(bins=bin_values, figsize=[8,4])




In [ ]:
dfSimTopic1_80[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
bin_values = np.arange(start=0.1, stop=1, step=0.1)
dfSimTopic2_80[1].hist(bins=bin_values, figsize=[8,4])



In [ ]:
dfSimTopic3_80[1].hist(bins=bin_values, figsize=[8,4])